In [6]:
import pickle
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from operator import itemgetter
THREADS = 16
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import time
import torch.nn.functional as F
loss_func = torch.nn.MSELoss()

In [7]:
trainset = pd.read_csv('train.csv')
testset = pd.read_csv('test.csv')
def traite_train_test(df):
    df['actors'] = df['actors'].apply(lambda x: json.loads(x))
    df['director'] = df['director'].apply(lambda x: json.loads(x))
    df['genre'] = df['genre'].apply(lambda x: json.loads(x))
    return df
hehe_test = traite_train_test(trainset)
df_empty = traite_train_test(testset)
df_empty['user_id'] = df_empty['user_id'].astype('int')
df_empty['user_rating'] = df_empty['user_rating'].astype('float')
df_empty['movie'] = df_empty['movie'].astype('int')
hehe_test.index = range(len(hehe_test))
df_empty.index = range(len(df_empty))

In [8]:
df_all = df_empty.append(hehe_test)

<ipython-input-8-55be2c786f61>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_empty.append(hehe_test)


In [9]:
max(hehe_test['user_id']),max(hehe_test['movie'])

(942, 1238)

In [10]:
def basic_fm(BIAS, bias, feature_embeddings):
    inter1 = torch.square(feature_embeddings.sum(0))
    inter2 = torch.square(feature_embeddings).sum(0)
    out = 0.5 * (inter1 - inter2)
    out = out.sum()
    out = BIAS + bias + out
    return out

In [11]:
def jiumin(first, user_item_id, actors_id, directors_id, genres_id, aspects, user_item_aspect, bias, BIAS):
    val_base = actors_id[first]
    actor_base = torch.LongTensor(val_base)
    bias_actors = bias(actor_base).squeeze(1).sum()
    actors_base = aspects(actor_base)

    val_base = directors_id[first]
    director_base = torch.LongTensor(val_base)
    bias_directors = bias(director_base).squeeze(1).sum()
    director_base = aspects(director_base)

    val_base = genres_id[first]
    genre_base = torch.LongTensor(val_base)
    bias_genres = bias(genre_base).squeeze(1).sum()
    genre_base = aspects(genre_base)

    val_base = user_item_id[first]
    user_item_base = torch.LongTensor(val_base)
    bias_user_items = bias(user_item_base).squeeze(1).sum()  # 1* n * 1
    user_item_base = user_item_aspect(user_item_base)  # 1 * 2 * dim

    bias_all = bias_actors + bias_directors + bias_genres + bias_user_items

    batch_embedding = torch.cat(
        (actors_base, director_base, genre_base, user_item_base), 0)

    pre_rating = basic_fm(BIAS, bias_all, batch_embedding)
    return pre_rating

In [12]:
class FM(nn.Module):
    def __init__(self, n_user_item, n_features, n_factors):
        super(FM, self).__init__()
        self.n_features = n_features
        self.n_factors = n_factors
        self.features_factors = torch.nn.Embedding(n_features, n_factors)
        self.user_item_factors = torch.nn.Embedding(n_user_item, n_factors)
        self.bias = torch.nn.Embedding(n_features, 1)
        self.BIAS = torch.nn.Embedding(1, 1)

    def forward(self, user_item_id, actors_id, directors_id, genres_id):

        aspects = self.features_factors
        user_item_aspect = self.user_item_factors
        bias = self.bias
        bias = self.bias
        BIAS = self.BIAS.weight.squeeze(1)

        first = actors_id.index[0]
        pre_rating = jiumin(
            first, user_item_id, actors_id, directors_id, genres_id, aspects, user_item_aspect, bias, BIAS)
        for i in actors_id.index[1:]:
            actors_f = jiumin(i, user_item_id, actors_id,
                              directors_id, genres_id, aspects, user_item_aspect, bias, BIAS)
            pre_rating = torch.cat((pre_rating, actors_f))
        return pre_rating

In [16]:
data = hehe_test.copy()
i = 0
batch_size = 2
num_example = 86572
indices = list(range(num_example))

model_test = FM(2182, 4328, 32)
aspects = model_test.features_factors
user_item_aspect = model_test.user_item_factors
bias = model_test.bias
bias = model_test.bias
BIAS = model_test.BIAS.weight.squeeze(1)


indexs = indices[i:min(i+batch_size, num_example)]
user_item_id = data.iloc[:, 0:2].loc[indexs].values
actors_id = data.iloc[:, 2].loc[indexs]
actors_id.index = range(len(actors_id))
directors_id = data.iloc[:, 3].loc[indexs]
directors_id.index = range(len(directors_id))
genres_id = data.iloc[:, 4].loc[indexs]
genres_id.index = range(len(genres_id))

In [17]:
first = actors_id.index[0]

In [18]:
(0.3788-0.2908)/0.2908

0.3026134800550207

In [19]:
bias_actors,bias_directors,bias_genres,bias_user_items,bias_all

NameError: name 'bias_actors' is not defined

In [33]:
# actors_base.shape,director_base.shape,genre_base.shape,user_item_base.shape,batch_embedding.shape

(torch.Size([5, 32]),
 torch.Size([1, 32]),
 torch.Size([3, 32]),
 torch.Size([2, 32]),
 torch.Size([11, 32]))

In [20]:
val_base = actors_id[first]
actor_base = torch.LongTensor(val_base)
bias_actors = bias(actor_base).sum()
actors_base = aspects(actor_base)

val_base = directors_id[first]
director_base = torch.LongTensor(val_base)
bias_directors = bias(director_base).sum()
director_base = aspects(director_base)

val_base = genres_id[first]
genre_base = torch.LongTensor(val_base)
bias_genres = bias(genre_base).sum()
genre_base = aspects(genre_base)

val_base = user_item_id[first]
user_item_base = torch.LongTensor(val_base)
bias_user_items = bias(user_item_base).squeeze(1).sum()  # 1* n * 1
user_item_base = user_item_aspect(user_item_base)  # 1 * 2 * dim

bias_all = bias_actors + bias_directors + bias_genres + bias_user_items

batch_embedding = torch.cat(
    (user_item_base,actors_base, director_base, genre_base), 0)

In [21]:
bias_actors,bias_directors,bias_genres,bias_user_items,bias_all

(tensor(3.2905, grad_fn=<SumBackward0>),
 tensor(-0.4905, grad_fn=<SumBackward0>),
 tensor(2.0874, grad_fn=<SumBackward0>),
 tensor(-0.1492, grad_fn=<SumBackward0>),
 tensor(4.7382, grad_fn=<AddBackward0>))

In [22]:
actors_base.shape,director_base.shape,genre_base.shape,user_item_base.shape,batch_embedding.shape

(torch.Size([5, 32]),
 torch.Size([1, 32]),
 torch.Size([3, 32]),
 torch.Size([2, 32]),
 torch.Size([11, 32]))

In [23]:
batch_embedding.shape

torch.Size([11, 32])

In [14]:
batch_embedding.sum(1).shape

torch.Size([11])

In [24]:
inter1 = torch.square(batch_embedding.sum(0))
inter2 = torch.square(batch_embedding).sum(0)
inter1.shape

torch.Size([32])

In [25]:
inter1 = torch.square(batch_embedding.sum(0))
inter2 = torch.square(batch_embedding).sum(0)
out = 0.5 * (inter1 - inter2)
out = out.sum()
out = BIAS + bias_all + out

In [66]:
def RMSE(data, model):
    user_item_index = data.iloc[:, 0:2].values
    actors_id = data.iloc[:, 2] 
    directors_id = data.iloc[:, 3]
    genres_id = data.iloc[:, 4]
    rating = torch.FloatTensor(
        data.iloc[:, 5].values).to(DEVICE)
    prediction = model(user_item_index, actors_id, directors_id, genres_id)
    rmse = loss_func(prediction, rating)
    mae = torch.nn.L1Loss()(prediction, rating)
    
    return rmse ** 0.5,mae

In [67]:
# hehe_test['movie'] = hehe_test['movie'] + 943
# df_empty['movie'] = df_empty['movie'] + 943
#df_all['movie'] = df_all['movie'] + 943

In [68]:
max(df_all['movie'])

1238

In [74]:
def train(lr, dim, reg, batch_size, num_epochs, data, test):
    model = FM(2182, 4328, dim).to(DEVICE)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr,weight_decay=reg)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, threshold_mode='abs',threshold = 0.005)
    for epoch in range(num_epochs):
        model.train()
        t1 = time.time()
        num_example = len(data)
        indices = list(range(num_example))
        for i in range(0, num_example, batch_size):
            optimizer.zero_grad()
            indexs = indices[i:min(i+batch_size, num_example)]
            users_index = data.iloc[:, 0:2].loc[indexs].values
            actors_id = data.iloc[:, 2].loc[indexs]
            actors_id.index = range(len(actors_id))
            directors_id = data.iloc[:, 3].loc[indexs]
            directors_id.index = range(len(directors_id))
            genres_id = data.iloc[:, 4].loc[indexs]
            genres_id.index = range(len(genres_id))
            rating = torch.FloatTensor(
                data.iloc[:, 5].loc[indexs].values).to(DEVICE)
            prediction = model(
                users_index, actors_id, directors_id, genres_id)

            err = loss_func(prediction, rating) 
            err.backward()
            optimizer.step()
        t2 = time.time()
        rmse, mae = RMSE(test, model)
        scheduler.step(rmse)
        if (epoch + 1) % 50 == 0:
            print("Epoch: ", epoch, " Loss: ", err, " RMSE in test set:",
              rmse, "MAE in test set: ", mae)
            print("Time consumed is:", t2-t1)
    return rmse, mae, model

In [75]:
def grid(lrs, dims, regs, batch_sizes, num_epochs, data, test):
    res = []
    models = []
    qnm = []
    for lr in lrs:
        for dim in dims:
            for reg in regs:
                for batch_size in batch_sizes:
                    rmse, mae, model = train(
                        lr, dim, reg, batch_size, num_epochs, data, test)
                    res.append(rmse)
                    res.append(mae)

                    models.append(model)
                    state = { 'model': model.state_dict()}   
                    torch.save(state, 'fm/fm_' + str(lr) + '_' + str(dim) + '_' + str(reg) + '_' + str(batch_size) + '.pkl')
                    
    return res, models

In [76]:
# lr = 1e-2
# dim = 32
# reg = 4e-5
# batch_size = 128 
# num_epochs = 100
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [80]:
from tqdm import tqdm
from torch.nn import LeakyReLU
# model_fm = train(lr, dim, reg, batch_size, num_epochs, hehe_test,df_empty)

In [82]:
rate = 0.0
lrs = [1e-2,5e-2]
dims = [16,32,64]
regs = [1e-6,1e-7]
batch_sizes = [128,256]
num_epochs = 150
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
res_reg, models_reg = grid(lrs,dims,regs,batch_sizes,num_epochs,hehe_test,df_empty)

Epoch:  49  Loss:  tensor(0.6841, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(1.3895, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1058, grad_fn=<L1LossBackward0>)
Time consumed is: 57.71477675437927
Epoch:  99  Loss:  tensor(0.0473, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.7127, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.5701, grad_fn=<L1LossBackward0>)
Time consumed is: 56.713467836380005
Epoch:  149  Loss:  tensor(0.0356, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.6522, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.5130, grad_fn=<L1LossBackward0>)
Time consumed is: 59.34599781036377
Epoch:  49  Loss:  tensor(0.2352, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(1.0757, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.8539, grad_fn=<L1LossBackward0>)
Time consumed is: 58.28889465332031
Epoch:  99  Loss:  tensor(0.0833, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.5882, grad_fn=<PowBackward0>) MAE in test se

Epoch:  149  Loss:  tensor(4.8116, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(4.2690, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.0072, grad_fn=<L1LossBackward0>)
Time consumed is: 49.80368185043335
Epoch:  49  Loss:  tensor(1021.7432, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(33.6661, grad_fn=<PowBackward0>) MAE in test set:  tensor(25.5053, grad_fn=<L1LossBackward0>)
Time consumed is: 47.47133255004883
Epoch:  99  Loss:  tensor(0.4313, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(2.2191, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.5319, grad_fn=<L1LossBackward0>)
Time consumed is: 49.11369967460632
Epoch:  149  Loss:  tensor(0.6336, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(1.6353, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1734, grad_fn=<L1LossBackward0>)
Time consumed is: 48.98140740394592
Epoch:  49  Loss:  tensor(311.7751, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(20.2544, grad_fn=<PowBackward0>) MAE in

KeyboardInterrupt: 

In [83]:
 #conding=utf8  
import os 

path = "fm/" 
def get_file_name(path):
    for file in os.listdir(path):
        lr = file.split('_')[1]
        dim = file.split('_')[2]
        reg = file.split('_')[3]
        batch_size = file.split('_')[4][:3]
        
        model = FM(2182, 4328, int(file.split('_')[2])).to(DEVICE)
        load_params = torch.load('fm/' + file)['model']
        model_params = model.state_dict()
        same_parsms = {k: v for k, v in load_params.items() if k in model_params.keys()}
        model_params.update(same_parsms)

        model.load_state_dict(model_params)
        rmse, mae = RMSE(df_empty, model)
        print("Learning rate: ",lr, "dimension of vector: ", dim, "Regularization rate: ", reg,"Batch_size: ", batch_size)
        print(" RMSE in test set:",
              rmse, "MAE in test set: ", mae)

In [84]:
get_file_name(path)

Learning rate:  0.1 dimension of vector:  16 Regularization rate:  1e-06 Batch_size:  256
 RMSE in test set: tensor(13.6390, grad_fn=<PowBackward0>) MAE in test set:  tensor(9.8624, grad_fn=<L1LossBackward0>)
Learning rate:  0.05 dimension of vector:  16 Regularization rate:  1e-07 Batch_size:  256
 RMSE in test set: tensor(3.9291, grad_fn=<PowBackward0>) MAE in test set:  tensor(2.8382, grad_fn=<L1LossBackward0>)
Learning rate:  0.001 dimension of vector:  16 Regularization rate:  1e-06 Batch_size:  128
 RMSE in test set: tensor(1.3709, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.0858, grad_fn=<L1LossBackward0>)
Learning rate:  0.01 dimension of vector:  16 Regularization rate:  1e-07 Batch_size:  128
 RMSE in test set: tensor(0.6411, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4951, grad_fn=<L1LossBackward0>)
Learning rate:  0.001 dimension of vector:  16 Regularization rate:  1e-07 Batch_size:  128
 RMSE in test set: tensor(1.4313, grad_fn=<PowBackward0>) MAE in test s